In [1]:
import os
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import soundfile
from IPython.display import clear_output

In [2]:
tp = pd.read_csv('kaggle-data/train_tp.csv')

In [62]:
# just get the song
for i in range(len(tp)):
    sig, sr = librosa.load('kaggle-data/train/%s.flac' % tp.recording_id[i])
    
    if sr != 22050:
        raise Exception('Sample rate is not 22050')
        
    smin = int(tp.t_min[i] * 22050)
    smax = int(tp.t_max[i] * 22050)
    species = tp.species_id[i]
    songtype = tp.songtype_id[i]

    output_filename = 'clips/s%02d-%01d/song%04d.wav' % (species, songtype, i)
    
    print(i, smax - smin, output_filename)
    
    # uncomment to actually write files
    #soundfile.write(output_filename, sig[smin:smax], 22050)

0 12936 clips/s14-1/song0000.wav
1 134145 clips/s23-1/song0001.wav
2 69149 clips/s12-1/song0002.wav
3 83327 clips/s17-4/song0003.wav
4 53888 clips/s10-1/song0004.wav
5 90903 clips/s08-1/song0005.wav
6 17524 clips/s00-1/song0006.wav
7 15290 clips/s18-1/song0007.wav
8 73735 clips/s15-1/song0008.wav
9 19757 clips/s01-1/song0009.wav
10 47275 clips/s06-1/song0010.wav
11 53888 clips/s10-1/song0011.wav
12 69268 clips/s07-1/song0012.wav
13 17521 clips/s00-1/song0013.wav
14 25089 clips/s11-1/song0014.wav
15 25088 clips/s11-1/song0015.wav
16 13759 clips/s13-1/song0016.wav
17 69148 clips/s12-1/song0017.wav
18 40455 clips/s16-4/song0018.wav
19 40455 clips/s16-4/song0019.wav
20 40455 clips/s16-4/song0020.wav
21 17521 clips/s00-1/song0021.wav
22 15287 clips/s18-1/song0022.wav
23 34221 clips/s19-1/song0023.wav
24 57507 clips/s17-4/song0024.wav
25 69265 clips/s07-1/song0025.wav
26 47275 clips/s06-1/song0026.wav
27 16345 clips/s03-1/song0027.wav
28 174720 clips/s23-4/song0028.wav
29 174720 clips/s23-4/

In [ ]:
# get up to 3 seconds of context, randomly adding before or after
rng = np.random.default_rng(seed = 42)

for i, row in tp.iterrows():
    input_filename = 'kaggle-data/train/%s.flac' % row.recording_id
    output_filename = 'clips30s/s%02d-%01d/song%04d.wav' % (row.species_id, row.songtype_id, i)
    smin = int(row.t_min * 22050)
    smax = int(row.t_max * 22050)
    
    sig, _ = librosa.load(input_filename)

    target_len = 3*22050
    if smax - smin < target_len:
        # start >= 0, end <= len, end >= smax, start <= smin
        lower = max(0, smax - target_len)
        upper = min(len(sig) - target_len, smin)
        start = rng.integers(lower, upper)
        end = start + target_len
    else:
        start = rng.integers(smin, smax - target_len)
        end = start + target_len

    clear_output(wait = True)
    print('%d / %d %s [%.1f,%.1f] -> [%.1f,%.1f]' % (i, len(tp), input_filename, 
        smin / 22050, smax / 22050, start / 22050, end/22050))
    
    # uncomment to actually write files
    soundfile.write(output_filename, sig[start:end], 22050)

863 / 1216 kaggle-data/train/b65e327ee.flac [22.1,23.3] -> [21.7,24.7]


In [ ]:
# get the song with full context
for i, row in tp.iterrows():
    input_filename = 'kaggle-data/train/%s.flac' % row.recording_id
    output_filename = 'clips_with_context/s%02d-%01d/song%04d.wav' % (row.species_id, row.songtype_id, i)
    smin = int(row.t_min * 22050)
    smax = int(row.t_max * 22050)
    
    sig, _ = librosa.load(input_filename)

    target_len = 3*22050
    start = min(smin, max(smax - target_len, 0))
    end = max(smax, min(smin + target_len, len(sig)))

    clear_output(wait = True)
    print('%d / %d %s [%.1f,%.1f] -> [%.1f,%.1f]' % (i, len(tp), input_filename, 
        smin / 22050, smax / 22050, start / 22050, end/22050))
    
    # uncomment to actually write files
    soundfile.write(output_filename, sig[start:end], 22050)

627 / 1216 kaggle-data/train/7e370f1f3.flac [46.1,46.7] -> [43.7,49.1]
